# Demo the Liquidator Agent Logic

This notebook:

1. Fetches the latest crvusd contract data into Python objects using `crvusdsim`.
2. Fetches prices and `ExternalMarket`s from the `baseline` scenario.
3. Instantiates a liquidator agent.
4. Checks if there are users to liquidate in the `Controller` and liquidates them.

In [1]:
from crvusdsim.pool import get
from src.sim.scenario import Scenario

%load_ext autoreload
%autoreload 2

In [2]:
ETH_POOL = "weth"
sim_market = get(ETH_POOL, bands_data="controller")
controller = sim_market.controller
stableswap_pools = sim_market.stableswap_pools

In [3]:
# Generate markets and prices
scenario = Scenario("baseline")
sample = scenario.pricepaths[0]
scenario.update_market_prices(sample)

markets = scenario.markets
liquidator = scenario.liquidator

liquidator.set_paths(controller, stableswap_pools, markets)

[DEBUG][23:25:24][root]-640975: Using 502560 1Inch quotes.


In [4]:
# Artificially inflate collateral price
collateral = controller.COLLATERAL_TOKEN.address
sample = scenario.pricepaths[0]
prices_usd = sample.prices_usd
assert collateral in prices_usd.keys()
prices_usd[collateral] *= 2
sample.update(prices_usd)
scenario.update_market_prices(sample)

In [5]:
profits, underwater_debt = liquidator.perform_liquidations(controller)

[INFO][23:25:26][root]-640975: There are 4 users to liquidate.
[INFO][23:25:26][root]-640975: Liquidating user 0x64495f323a368b5bb3b3c80053622640299b331d with expected profit: 34245.461988.
[INFO][23:25:26][root]-640975: Executing cycle Cycle(Trades: [Swap(pool=Curve.fi Factory Plain Pool: crvUSD/USDC, in=USDC, out=Curve.Fi USD Stablecoin (crvUSD), amt=49709425614), Liquidation(controller=<crvusdsim.pool.sim_interface.sim_controller.SimController object at 0x7fe455f6da90>, position=<crvusdsim.pool.crvusd.controller.Position object at 0x7fe437141010>, amt=49831440353605447581696, frac=1000000000000000000, i=0, j=1), Swap(pool=External Market (USDC, WETH), in=Wrapped Ether, out=USD Coin, amt=20131821106939953152)], Expected Profit: 34245.461988).
[INFO][23:25:26][root]-640975: Executing trade Swap(pool=Curve.fi Factory Plain Pool: crvUSD/USDC, in=USDC, out=Curve.Fi USD Stablecoin (crvUSD), amt=49709425614).
[INFO][23:25:26][root]-640975: Executing trade Liquidation(controller=<crvusdsim.